Загрузка нужных версий библиотек

In [ ]:
!pip install torch==1.11.0
!pip install torchvision==0.12.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.7 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu118
    Uninstalling torch-2.1.0+cu118:
      Successfully uninstalled torch-2.1.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 17.4 MB/s eta 0:00:00


In [ ]:
!pip install pycocotools

In [ ]:
%%shell

# Download Torch/Vision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.8.2

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

In [ ]:
import os
import torch
import torchvision
import numpy as np
import pandas as pd
import cv2
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from engine import train_one_epoch, evaluate

Загружаем данные с гугл-диска.


In [ ]:
from google.colab import drive


drive.mount('/content/drive')
id="/content/drive/MyDrive/hack1"

In [ ]:
train_csv_path = id+"/data.csv"

train_images = id


In [ ]:
train_data=pd.read_csv(train_csv_path)
train_data

Класс C_Dataset нужен для обучения модели

In [ ]:
import math
from torchvision import transforms

class C_Dataset(torch.utils.data.Dataset):
    def __init__(self, df, image_path,**kwargs):
        self.images_p = df["filename"].to_numpy()
        self.df = df.drop(columns=["filename"])
        self.image_path = image_path
        self.targets=[]
        self.images=[]
        self.F = transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    ToTensorV2(p=1)
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))#transforms.ToTensor()
        self.transforms=transforms
        for i in range(len(df)):
          if(self.images_p[i][-4:]!=".png"):
            self.images_p[i]=self.images_p[i]+".png"
        df=df.drop(columns=["filename"])
        data=df.to_numpy()
        for i in range(len(data)):
          print(image_path+"/"+self.images_p[i])
          img = cv2.imread(image_path+"/"+self.images_p[i])
          w=0
          h=0
          h=len(img)
          w=len(img[0])
          l=0
          for j in range(len(data[i])):
            if not(np.isnan(data[i][j])):
              l+=1
          l//=5
          xmins = []
          ymins = []
          xmaxs = []
          ymaxs = []
          labels=[]
          count=0
          for j in range(l):
            count+=1
            x_m=max(min(data[i][5*j+1], data[i][5*j+2+1]),0)
            x_ma=min(max(data[i][5*j+1], data[i][5*j+2+1]), w)
            y_m=max(min(data[i][5*j+2], data[i][5*j+4]),0)
            y_ma=min(max(data[i][5*j+2], data[i][5*j+4]), h)
            labels.append(int(data[i][5*j]))
            xmins.append(x_m)
            xmaxs.append(x_ma)
            ymins.append(y_m)
            ymaxs.append(y_ma)
          for j in range(len(xmins)):
            if xmins[j]<0:
              xmins[j]=0
            if ymins[j]<0:
              ymins[j]=0
          labels = torch.as_tensor(labels, dtype=torch.int64)
          boxes = torch.as_tensor(np.stack([xmins, ymins, xmaxs, ymaxs], axis=1), dtype=torch.float32)
          areas = (boxes[:,3] - boxes[:,1]) * (boxes[:,2] - boxes[:,0])
          areas = torch.as_tensor(areas, dtype=torch.float32)
          image_id = torch.tensor([i])
          iscrowd = torch.zeros((len(labels),), dtype=torch.int64)
          target = {}
          target["boxes"] = boxes
          target["labels"] = labels
          target["image_id"] = image_id
          target["area"] = areas
          target["iscrowd"] = iscrowd
          self.targets.append(target)
          self.images.append(image_path+"/"+self.images_p[i])
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
              img=cv2.imread(self.images[idx])
              img = self.F(image=img, bboxes=self.targets[idx]["boxes"], labels=self.targets[idx]["labels"])
              return torch.as_tensor(img["image"], dtype=torch.float32), self.targets[idx]
    def get_height_and_width(self, image):
        image_data = self.df.loc[self.df['filename'] == image]
        return image_data["width"].values[0], image_data["height"].values[0]
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
train_dataset = C_Dataset(train_data, id)


In [ ]:
data_loader_train = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)


Используем алгоритм Faster RCNN с опорной моделью ResNet50

При этом модель распознаёт 4 класса пыли

In [ ]:
detection_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 4
in_features = detection_model.roi_heads.box_predictor.cls_score.in_features
detection_model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

Указываем, что обучение модели будет происходить на GPU для экономии времени

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
detection_model.to(device)

In [ ]:
def training(model, train_loader, val_loader, epochs=10):
    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.00001, momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)
    for epoch in range(epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=1)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        #evaluate(model, val_loader, device=device)

Обучаем модель и сохраням веса

In [ ]:
training(detection_model, data_loader_train,val_loader=data_loader_train, epochs=30)
torch.save(detection_model.state_dict(), id+'/resnet1_50.pth')